In [1]:
import pandas as pd
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import gensim
from sacremoses import MosesDetokenizer


In [2]:
pol = pd.read_csv("Policy.csv", engine = "python", names = ["Policy", "Text"])
non_pol = pd.read_csv("No_Policy - Corpus.csv", engine = "python", names = ["Policy", "Text"])
df = pd.DataFrame.append(pol, non_pol)
#df = df.sample(frac=1).reset_index(drop=True)
df['Policy'].loc[df['Policy'] == -1] = 0

C:\Users\Djordje\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
df['char_count'] = df['Text'].str.len()
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split(" ")))
df.groupby(['Policy']).mean()

,char_count,word_count
Policy,,
0,4356.092105,604.25
1,22816.780000,3635.58


In [4]:
df.mean()

Policy           0.247525
char_count    8925.569307
word_count    1354.579208
dtype: float64

In [5]:
freq_tot = pd.Series(' '.join(df['Text']).split()).value_counts()[:10]
freq_pol = pd.Series(' '.join(pol['Text']).split()).value_counts()[:10]
print(freq_tot, freq_pol)

to             9127
the            8239
and            7636
you            5659
or             5267
of             5173
your           4986
information    3880
a              3508
in             3007
dtype: int64 to             6716
and            5519
the            5472
you            4573
your           3926
or             3911
of             3527
information    3423
may            2150
a              2117
dtype: int64


In [6]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))   
    return result

In [7]:
df['Text'] = df['Text'].map(preprocess)
pol['Text'] = pol['Text'].map(preprocess)

In [8]:
mdtk = MosesDetokenizer()
df['Text'] = df['Text'].map(mdtk.detokenize)
pol['Text'] = pol['Text'].map(mdtk.detokenize)

df['char_count'] = df['Text'].str.len()
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split(" ")))
df.groupby(['Policy']).mean()

,char_count,word_count
Policy,,
0,2395.447368,351.993421
1,11457.840000,1657.240000


In [9]:
freq_tot = pd.Series(' '.join(df['Text']).split()).value_counts()[:10]
freq_pol = pd.Series(' '.join(pol['Text']).split()).value_counts()[:10]
print(freq_tot, freq_pol)

inform     5497
servic     2861
person     1698
privaci    1599
provid     1491
account    1347
cooki      1301
collect    1254
parti      1171
polici     1133
dtype: int64 inform     4829
servic     2398
person     1414
privaci    1260
provid     1221
collect    1122
account    1079
cooki       941
parti       935
polici      905
dtype: int64


In [10]:
df.mean()

Policy           0.247525
char_count    4638.613861
word_count     675.074257
dtype: float64